<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [90]:
%load_ext autoreload
%autoreload 2

from bs4 import BeautifulSoup
import requests
from bs4 import Comment, Tag
import pandas as pd
import bs4
from dictionaries import *
PATH = '/Users/doguaraci/Desktop/nba'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [55]:
def get_html(link):
    source = requests.get(link).text
    soup = BeautifulSoup(source,'html5lib') 
    
    return soup

def get_table_html(soup):
    for comment in soup.find_all(text=lambda e: isinstance(e, bs4.Comment)):
        tag = bs4.Tag(name="comment")
        tag.string = comment.strip()
        comment.replace_with(tag)
    a = soup.find_all('comment')
    text = str(a[21])
    text = text.replace('&lt;','<')
    text = text.replace('&gt;','>')
    
    result = BeautifulSoup(text,'html5lib') 
    
    return result.find('table')

def get_match_list(soup):
    games = []
    for a in soup.find_all('tr')[1:]:    
        newdict = {}
        for d in a.find_all('td'):
            newdict[d['data-stat']] = d.text
            games.append(newdict)   
    return games

def create_df(playoffs):
    df = pd.DataFrame(playoffs)
    columns = df.columns
    final = df.drop_duplicates().reset_index()[['age', 'ast', 'blk', 'date_game', 'drb', 'fg', 'fg3', 'fg3_pct', 'fg3a',
        'fg_pct', 'fga', 'ft', 'ft_pct', 'fta', 'game_location', 'game_result',
        'game_score', 'game_season', 'gs', 'mp', 'opp_id', 'orb', 'pf', 'pts',
        'stl', 'team_id', 'tov', 'trb']]
    
    return final

In [56]:
def get_playoff_df(link):

    soup = get_html(link)
    soup = get_table_html(soup)
    playoffs = get_match_list(soup)

    return create_df(playoffs)

In [57]:
link = 'https://www.basketball-reference.com/players/c/curryst01/gamelog/2015'
df = get_playoff_df(link)

In [58]:
def get_choke(df):
    if (df.game_score<12 and df.difference<15):
        return 1
    else:
        return 0

def process_df(df):

    df['result'] = df.game_result.apply(lambda x: x[0])

    df['difference'] = df.game_result.apply(lambda x: x[3:-1])
    df['difference'] = df.difference.apply(int)
    df['game_score'] = df.game_score.apply(float)

    df['W'] = 0
    df['L'] = 0
    df['stage'] = 0
    
    df['choke'] = df.apply(get_choke, axis=1)
    df['great'] = df.game_score.apply(lambda x: 1 if x>=24 else 0)
    df['monster'] = df.game_score.apply(lambda x: 1 if x>=30 else 0)

    for i in range(df.shape[0]):
        if i == 0:
            df.loc[i,'stage'] = 1

        else:
            if df.loc[i,'opp_id'] == df.loc[i-1,'opp_id']:
                df.loc[i,'stage'] = df.loc[i-1,'stage'] 
                if df.loc[i-1,'result'] == 'W':
                    df.loc[i,'W'] = df.loc[i-1,'W'] + 1
                    df.loc[i,'L'] = df.loc[i-1,'L']
                else: 
                    df.loc[i,'L'] = df.loc[i-1,'L'] + 1
                    df.loc[i,'W'] = df.loc[i-1,'W']
            else:
                df.loc[i,'stage'] = df.loc[i-1,'stage'] + 0.33
    
    def weighted(df):
        return df.game_score * df.stage
    
    df['w_game_score'] = df.apply(weighted,axis=1)

    criteria = ((df["W"] == 0) & (df["L"] == 2)) \
                | ((df["W"] == 1) & (df["L"] == 2)) \
                | ((df["W"] == 1) & (df["L"] == 3)) \
                | ((df["W"] == 2) & (df["L"] == 2)) \
                | ((df["W"] == 2) & (df["L"] == 3)) \
                | ((df["W"] == 3) & (df["L"] == 2)) \
                | ((df["W"] == 3) & (df["L"] == 3)) 

    return df.loc[criteria]

In [60]:
def get_player_results(name):
    yearlist = player2year[name]
    player = []
    for year in yearlist:
        try:
            link = "https://www.basketball-reference.com/players/c/" + player2code[name] +"/gamelog/" + year
            df = get_playoff_df(link)
            df = process_df(df)
            df['year'] = year
            player.append(df)
        except:
            continue
    result = pd.concat(player).reset_index()
    result['name'] = name
    
    return {'Name': name, \
            'Average':round(result.game_score.mean(),2), \
            'Weighted Average':result.w_game_score.sum() / result.stage.sum(), \
            'Choke_count':result.choke.sum(), \
            'Great_count':result.great.sum(), \
            'Monster_count':result.monster.sum(), \
            'Win_percent':round(result[result['result']=='W'].shape[0] / result.shape[0],2), \
            'Total_games': result.shape[0]}, result

In [92]:
allplayer_scores = []
allplayer_games = []
for player in player2year.keys():
    score, games = get_player_results(player)
    allplayer_scores.append(score)
    allplayer_games.append(games)
    print("Done: {}".format(player))

scores_df = pd.DataFrame(allplayer_scores).reset_index()
games_df = pd.concat(allplayer_games).reset_index()

Done: James Harden
Done: Stephen Curry
Done: Anthony Davis
Done: Kevin Durant
Done: LeBron James
Done: Russell Westbrook
Done: Kawhi Leonard
Done: Chris Paul
Done: Kobe Bryant
Done: Tim Duncan
Done: Dwight Howard
Done: Dwayne Wade
Done: Dirk Nowitzki
Done: Kevin Garnett
Done: Steve Nash
Done: Allen Iverson
Done: Shaquille O'Neal
Done: Tracy McGrady
Done: Jason Kidd
Done: Gary Payton
Done: Karl Malone
Done: David Robinson
Done: Scottie Pippen
Done: John Stockton
Done: Michael Jordan
Done: Charles Barkley
Done: Hakeem Olajuwon
Done: Patrick Ewing
Done: Dominique Wilkins
Done: Isiah Thomas
Done: Magic Johnson
Done: Larry Bird
